In [ ]:
# Imports
import os
import asyncio
from typing import cast
from dotenv import load_dotenv
from typing import Dict
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent


# Arize Phoenix Tracer
from phoenix.otel import register
from opentelemetry import trace as otel_trace


# Markdown output display
from IPython.display import Markdown, display

In [2]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [3]:
# Load Environment Variables
load_dotenv(override=True)

# Gemini Environment Variables
gemini_api_key = env_to_str('GEMINI_API_KEY')
gemini_base_url = env_to_str('GEMINI_BASE_URL')
model_gemini_flash = env_to_str('MODEL_GEMINI_FLASH')

# Phoenix Collector Variables
phoenix_collector_endpoint = env_to_str("PHOENIX_COLLECTOR_ENDPOINT") or "http://localhost:6006/v1/traces"

In [4]:
# Configure Phoenix Tracer
try:
    tracer_provider = register(
    project_name="gemini_phoenix",
    auto_instrument=True,
    endpoint=phoenix_collector_endpoint,
    set_global_tracer_provider=True
)
    display(Markdown("✅ Phoenix tracer registered successfully"))
except Exception as e:
    display(Markdown(f"❌ Phoenix registration failed: {e}"))

DependencyConflict: requested: "openai-agents >= 0.1.0" but found: "openai-agents 0.0.17"
DependencyConflict: requested: "google-genai" but found: "None"


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: gemini_phoenix
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



✅ Phoenix tracer registered successfully

In [5]:
# Gemini Model
gemini_model = OpenAIChatCompletionsModel(
    model=model_gemini_flash,
    openai_client=AsyncOpenAI(
        api_key=gemini_api_key, 
        base_url=gemini_base_url
    )
)

# Gemini Agent
gemini_agent = Agent(
    name="The Jokester", 
    instructions="You are a joke teller.", 
    model=gemini_model
)

In [6]:
# Test function with explicit tracing
async def gemini_tracer():
    tracer = otel_trace.get_tracer(__name__)
    with tracer.start_as_current_span("Chicken Joke"):
        
        # Add custom attributes to the trace
        current_span = otel_trace.get_current_span()
        current_span.set_attribute("user.request", "Tell a joke about chickens and roads")
        current_span.set_attribute("agent.name", "The Jokester")
        current_span.set_attribute("model.name", model_gemini_flash or "unknown")
        
        result = await Runner.run(gemini_agent, "Tell a joke about chickens and roads.")
        
        # Add response attributes
        current_span.set_attribute("response.length", len(result.final_output))
        current_span.set_attribute("response.preview", result.final_output[:100])
        
        display(Markdown(result.final_output))
        return result.final_output
 

In [7]:
result = await gemini_tracer()

Why did the chicken cross the playground?

To get to the other slide! 

... I know, I know, it's not the road one. But listen, that one's so overdone! I'm trying to reinvent the chicken-crossing genre! Maybe I'll try again...

Why did the chicken cross the road, roll in the mud, and cross back?

To be a dirty double-crosser!

... Okay, I'll work on my delivery. Maybe I should stick to eggs. They're always good for a yolk. Get it? Yolk? Ah, never mind.


[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: ollama_o*********ault. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}


**IMPORTANT:**
- **THIS EXAMPLE -DOES- USE TRACE**
- **ARIZE PHOENIX TRACE:**
  - **OLLAMA**
  - **GEMINI**